In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellek.qa.ablation import answer_question, answer_question_with_reasoning, answer_question_with_triplets, answer_question_reasoning_with_triplets
from bellek.utils import set_seed, jprint
from bellek.musique.singlehop import benchmark

set_seed(89)

/home/pc/anaconda3/envs/baris/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
from tqdm.auto import tqdm
tqdm.pandas()

In [4]:
from bellek.musique.constants import ABLATION_RECORD_IDS

df = pd.read_json('../../data/generated/musique-evaluation/dataset.jsonl', orient='records', lines=True)
df = df.set_index('id', drop=False).loc[ABLATION_RECORD_IDS].copy().reset_index(drop=True)
qd_df = pd.read_json('../../data/generated/musique-evaluation/question-decomposition.jsonl', orient='records', lines=True)
df = pd.merge(df.drop(columns=['question', 'question_decomposition']), qd_df, on='id', suffixes=('', ''))
# df = df.head(10)

print(df.shape)
df.head()

(100, 8)


,id,paragraphs,answer,answer_aliases,answerable,answers,question,question_decomposition
0,2hop__575188_342798,"[{'idx': 0, 'title': 'Liliana Mumy', 'paragrap...",Ahmad Shah Qajar,[Ahmad Shah Qajar],True,[Ahmad Shah Qajar],Who is the child of Mahmoud Mirza's father?,"[{'id': 575188, 'question': 'Who is Mahmoud Mi..."
1,2hop__731584_700117,"[{'idx': 0, 'title': 'KAPE', 'paragraph_text':...",Berrien County,[Berrien County],True,[Berrien County],In which county is the city to which KKVU is l...,"[{'id': 731584, 'question': 'To which city is ..."
2,2hop__690412_526810,"[{'idx': 0, 'title': 'Cabramatta Creek', 'para...",Chao Phraya River,[Chao Phraya River],True,[Chao Phraya River],For what river does the river on which Pa Sak ...,"[{'id': 690412, 'question': 'On which river is..."
3,2hop__263638_69048,"[{'idx': 0, 'title': 'Michael J. Barron', 'par...",Honorable Justice Abiodun Smith,[Honorable Justice Abiodun Smith],True,[Honorable Justice Abiodun Smith],Who is the Chief Judge of the Tebesa Nemine's ...,"[{'id': 263638, 'question': 'Where was Tebesa ..."
4,2hop__142842_68489,"[{'idx': 0, 'title': 'Perfect Night: Live in L...",Snapper Foster,[Snapper Foster],True,[Snapper Foster],Who did the performer of Night Rocker play on ...,"[{'id': 142842, 'question': 'Who performed Nig..."


In [5]:
perfect_retrieval_func = lambda docs, query: [doc for doc in docs if doc['is_supporting']]

In [6]:
N_RUNS = 3
results = []

In [7]:
for i in range(1, N_RUNS+1):
    df_standard, scores = benchmark(df, answer_question, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs", "qa": "standard", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.58,
  "f1": 0.6739117287940816,
  "fuzzy_match": 0.7
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.6,
  "f1": 0.6859117287940816,
  "fuzzy_match": 0.72
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.58,
  "f1": 0.6699117287940817,
  "fuzzy_match": 0.7
}


In [8]:
for i in range(1, N_RUNS+1):
    df_cot, scores = benchmark(df, answer_question_with_reasoning, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs", "qa": "cot", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.45,
  "f1": 0.5936246110211627,
  "fuzzy_match": 0.59
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.45,
  "f1": 0.5761277056277057,
  "fuzzy_match": 0.59
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.46,
  "f1": 0.5999610389610389,
  "fuzzy_match": 0.62
}


In [9]:
for i in range(1, N_RUNS+1):
    df_cte, scores = benchmark(df, answer_question_with_triplets, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs", "qa": "cte", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.59,
  "f1": 0.7154314764183185,
  "fuzzy_match": 0.73
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.58,
  "f1": 0.7056391647568119,
  "fuzzy_match": 0.71
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.59,
  "f1": 0.7224011733880156,
  "fuzzy_match": 0.73
}


In [10]:
for i in range(1, N_RUNS+1):
    df_cte_cot, scores = benchmark(df, answer_question_reasoning_with_triplets, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs", "qa": "cte+cot", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.53,
  "f1": 0.6636184764760477,
  "fuzzy_match": 0.68
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.5,
  "f1": 0.6489337474120082,
  "fuzzy_match": 0.69
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.55,
  "f1": 0.6791877156659766,
  "fuzzy_match": 0.7
}


# Report

In [11]:
report_df = pd.DataFrame.from_records(results, columns=['context', 'retrieval', 'qa', 'run', 'exact_match', 'f1'])
report_df

,context,retrieval,qa,run,exact_match,f1
0,paragraphs,groundtruth,standard,1,0.58,0.673912
1,paragraphs,groundtruth,standard,2,0.60,0.685912
2,paragraphs,groundtruth,standard,3,0.58,0.669912
3,paragraphs,groundtruth,cot,1,0.45,0.593625
4,paragraphs,groundtruth,cot,2,0.45,0.576128
5,paragraphs,groundtruth,cot,3,0.46,0.599961
6,paragraphs,groundtruth,cte,1,0.59,0.715431
7,paragraphs,groundtruth,cte,2,0.58,0.705639
8,paragraphs,groundtruth,cte,3,0.59,0.722401
9,paragraphs,groundtruth,cte+cot,1,0.53,0.663618


In [12]:
report_df.drop(columns=['context', 'retrieval', 'run']).groupby(['qa']).mean().loc[['standard', 'cot', 'cte', 'cte+cot']]

,exact_match,f1
qa,,
standard,0.586667,0.676578
cot,0.453333,0.589904
cte,0.586667,0.714491
cte+cot,0.526667,0.663913


In [24]:
i = 2
row = df_cot.iloc[i]
hop = row['raw_output']['hops'][0]
print(row['question'])
print(row['answers'])
print(hop['llm_output'].answer)
print(hop['llm_output'].reasoning)

For what river does the river on which Pa Sak Jolasid Dam is located serve as the mouth?
['Chao Phraya River']
Chao Phraya River
The Pa Sak Jolasid Dam impounds the Pa Sak River. The Pa Sak River is one of the principal tributaries of the Chao Phraya River in Thailand. Therefore, the river on which the Pa Sak Jolasid Dam is located serves as a tributary to the Chao Phraya River.
